### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Summer 2025 ###

# Optimized Meal Planning #

#### Student 1 (email address), Student 2 (email address), Student 3 (email address)

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

Meal planning is a widespread issue for people all over the world. It is a difficult balance of getting enough nutrients, not wasting food, and spending as little money as possible. As someone planning how to buy ingredients for weekly grocery shopping, the question is what ingredients do you need to buy and what recipes should you make.

Given a set of recipes that you know and a set of ingredients that are available to buy from the grocery store, how can grocery shopping be optimized to save money. Given that a kitchen stores food from the past, constraints involving the amount of food that can be stored and how long it can be stored must be considered for a realistic model. A proper grocery shopping plan will also cover all necessary nutrients in a given week.

Important progress in this problem has been made in the past. George Stigler, an American economist, addressed the issue of how to minimize the cost of fulfilling all of a person's nutrition requirements given a list of food with known nutritional values (https://en.wikipedia.org/wiki/Stigler_diet). This work was one of the earliest examples of an optimization problem. Stigler used a heuristic approach but developed the foundation for a problem that is solvable using modern optimization. This work is useful for people that go grocery shopping and wish to minimize the cost. It makes planning easier and more efficient.

This work builds off of Stigler's work by formalizing the problem into a linear program. It goes further to allow for meal planning across weeks, knowing that food can be stored in the fridge after being bought.

The model in this program will use a mix of data types. This model is highly customizable. Any recipes can be chosen but a set of recipes will be created by hand for the purpose of demonstration. Recipes will consist of fixed ratios of each ingredient in the database. These recipes will use real ingredients. The nutrition information for each ingredient can be found online (https://www.nutritionix.com/search?q=co). As grocery prices fluctuate, prices will be chosen based on a snapshot of current prices as of xx-xx-xxxx from a typical Wisconsin grocery store website (https://www.metromarket.net/). Estimates for volume taken up by each ingredient will be made by hand.

## 2. Mathematical model ##

A discussion of the modeling assumptions made in the problem (e.g. is it from physics? economics? something else?). Explain the decision variables, the constraints, and the objective function. Finally, show the optimization problem written in standard form. If converting the problem to standard form would create a difficult-to-read model, it is okay to leave it in its simplest original form. Discuss the model type (LP, QP, MIP, etc.). Equations should be formatted in $\\LaTeX$ within the IJulia notebook. For this section you may **assume the reader is familiar with the material covered in class**.

Here is an example of an equation:

$$\begin{bmatrix}
      1 & 2 \\
       3 & 4
    \end{bmatrix}
    \begin{bmatrix} x \\ y \end{bmatrix} =
    \begin{bmatrix} 5 \\ 6 \end{bmatrix}$$

And here is an example of an optimization problem in standard form:
$$\begin{aligned}
  \underset{x \in \mathbb{R^n}}{\text{maximize}}\qquad& f_0(x) \\
    \text{subject to:}\qquad& f_i(x) \le 0 && i=1,\dots,m\\
    & h_j(x) = 0 && j=1,\dots,r
    \end{aligned}$$


## 3. Solution ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I may run all or parts of your code, so make sure it works**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may want to solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

In [1]:
using JuMP, HiGHS

m = Model()

things = [:horses, :donkeys, :goats]  # these are the things
@variable(m, x[things] >= 0)          # the quantities of each of the things (can't be negative)
@constraint(m, sum(x) <= 10)          # we can't have any more than 10 things total
@objective(m, Max, x[:horses])        # we want to maximize the number of horses

set_silent(m) # supress solver output for a cleaner report
set_optimizer(m, HiGHS.Optimizer)

optimize!(m)

for i in things
    println("The total number of ", i, " is: ", value(x[i]))     # print result
end

The total number of horses is: 10.0
The total number of donkeys is: 0.0
The total number of goats is: 0.0


## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

 Use plots (see `Plots` examples from class), or you can display results in a table like this:

| Tables        | Are          | Cool  |
| ------------- |:-------------| -----:|
| col 3 is      |right-aligned |\$1600 |
|  colons       | align columns|  \$12 |
| zebra stripes |    are neat  |   \$1 |

### 4.A. Feel free to add subsections

**Issues/Concerns**
1. Many of the variables should be integers. Many ingredients can only be bought from a store in fixed amounts, thought ingredients can be kept as a fractional amount in the fridge/pantry. Recipes can be difficult to scale by fractional amounts, though scaling recipes by an integer is always straightforward. Two approaches that would help with these concerns are integer programming techniques and formulating the problem as a Min-Cost Network Flow Problem with unimodular data matrices.
2. It may make sense to add a tradeoff variable. The MAX recipes per week variable is a constraint that could be itself optimized. Same thing for the recipes. Substituting ingredients and tweaking the proportions are realistic things a cook do to account for dietary restrictions, ingredient scarcity, flavor preference, etc.
3. Ingredient costs may fluctuate over time. It may make sense to further complicate the data and change $C_i$ to $C_{ik} \ (i \in I, k \in T)$
4. Assuming 3 is implemented, it is not realistic to know every future price fluctuation in advance. Therefore it makes more sense to create a model that is blind to future conditions.

**Possible Approaches for concerns**
1. The next couple weeks of lecture may enlighten us on new techniques to approach problems with an integer constraint. It will take some mental work to re-imagine the problem as combinatorial, but that seems like a goal that can be reasonable achieved by the deadline. The fact that inventory can be stored in fractional amounts is already accounted for in the existing model.

2. This seems like less of a priority for our project than other issues, but I think this would be useful for an end user using our model.

3. As we move on to generating data with scripts rather that hand-typing, we will see how cost-change over time makes data-creation more complex.

4. This seems like a possible application of stochastic programming, which I know almost nothing about and may be beyond the scope of this class. I wonder if there is a way to force the model to only consider only the next $t \leq |T|$ time slices, using only constraints.

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.